In [ ]:
"""PLEASE RUN ALL CELLS IN THE REQUIREMENTS NOTEBOOK FIRST"""

# LitteLemon Analysis and Sales Report

## Task 1

Task 1:
Complete the following steps to establish a connection pool:

To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

To find the information on the error, import the Error class from MySQL Connector/Python.

Define your database configurations as a Python dictionary object called dbconfig.

Establish a connection pool [pool_name = pool_b] with two connections. 

Implement error handling using a try-except block in case the connection fails. 


In [ ]:
"""Importing Required Libraries. Please run all cells in the requirements jupyter notebook first before running this"""
from mysql.connector.pooling import MySQLConnectionPool as myconpool
from mysql.connector import Error

In [ ]:
"""Database configurations"""
dbconfig = {
    'host': "abbeytetteh.com",
    'user': 'abbeytet_abbey',
    'password': 'Emma12nuel!',
    'database': 'abbeytet_learner'
}

In [ ]:
"""Establish a connection pool"""
try:
    pool = myconpool(
        pool_name = "pool_b",
        pool_size = 2,
        **dbconfig
    )
    print(f"Pool created successfully with Pool name {pool.pool_name} and size {pool.pool_size}")
except Error as e:
    print(f"An error occured.\n Error Code: {e.errno}.\n Error Message: {e.msg}")

# Task 2
Task 2:
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:


TIP: You need to add a connection to connect the third guest.


Guest 1:

Table Number: 8

First Name: Anees

Last Name: Java

Booking Time: 18:00

EmployeeID: 6


Guest 2:

Table Number: 5

First Name: Bald

Last Name: Vin

Booking Time: 19:00

EmployeeID: 6


Guest 3:

Table Number: 12

First Name: Jay

Last Name: Kon

Booking Time: 19:30 

EmployeeID: 6


Return all the connections back to the pool. 


In [ ]:
"""Get connection from Connection Pool for first guest"""

connection1 = pool.get_connection()
if connection1.is_connected:
    print("Guest 1 is connected")
    cursor = connection1.cursor()

In [ ]:
"""Insert data for first guest"""


GuestOneInsert = """
insert into Bookings(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) values (8, "Anees", "Java", "18:00:00", 6)
"""

In [ ]:
print("Inserting Guest 1 Data")
cursor.execute(GuestOneInsert)
connection1.commit()
print("Guest Data inserted Successfully")


In [ ]:
connection2 = pool.get_connection()
if connection2.is_connected:
    print("Guest 2 is connected")
    cursor = connection2.cursor()
else:
    print("Error Connecting to Database")

In [ ]:
GuestTwoInsert = """
insert into Bookings(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) values (5, "Bald", "Vin", "19:00:00", 6)
"""

In [ ]:
print("Inserting Guest 1 Data")
cursor.execute(GuestTwoInsert)
connection2.commit()
print("Guest Data inserted Successfully")

In [ ]:
try:
    connection3 = pool.get_connection()
    if connection3.is_connected():
        print("Guest 3 is connected")
except Error as e:
    print(e.errno, e.msg)
    print("Connection pool is full, Adding new connection")
    pool.add_connection()
    print("A new connection has been added successfully")
    connection3 = pool.get_connection()
    print(f"Guest 3 is connected successfully")
    cursor = connection3.cursor()
    

In [ ]:
GuestThreeInsert = """
insert into Bookings(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) values (12, "Jay", "Kon", "19:30:00", 6)
"""

In [ ]:
print("Inserting Guest 3 Data")
cursor.execute(GuestThreeInsert)
connection3.commit()
print("Guest Data inserted Successfully")

In [ ]:
print("Returning connections to the pool")
try:
    print("Returning connection 1")
    connection1.close()
    print('Connection 1 returned successfully')
    
    print("Returning connection 2")
    connection2.close()
    print('Connection 2 returned successfully')
    
    print("Returning connection 3")
    connection3.close()
    print('Connection 3 returned successfully')
except Error as e:
    print(e.errno, e.msg)
    
    

# Task 3
Create a report containing the following information:

The name and EmployeeID of the Little Lemon manager.

The name and role of the employee who receives the highest salary.

The number of guests booked between 18:00 and 20:00.

The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.


In [ ]:
"""Get a connection from connection pool for Task 2"""

print("Get connection from pool")
task3connection = pool.get_connection()
if task3connection.is_conneced:
    print("Connected successfully")
else:
    print("Error getting connection")

In [ ]:
#create a cursor for task4
cursor = task3connection.cursor()

In [ ]:
#Select Manager Statement
ManagerDetails = """Select EmployeeID, Name from Employees where  Role = "Manager" """

In [ ]:
#Execute select statement
cursor.execute(ManagerDetails)

In [ ]:
#Fetch results
result = cursor.fetchall()

In [ ]:
#Display results
for data in result:
    print(f"""
    Little Lemon Manager
    Name: {data[1]}
    EmployeeID: {data[0]} 
    """)

## Important

For the next tasks, the MYSQL query are of two versions. One without the group by and the other with the group by.

The first is "select Name, Role, max(Annual_Salary from Employees" - This works for some versions of mysql and the version I used for this task. 

However it doesn't work for latest versions of my mysql. I read about the error and found out that I need to add a group by to my argument. Here is the link if you want to find out more 
https://stackoverflow.com/questions/43481869/aggregated-query-without-group-by

I have however included the group by in the next cell, To be able to run the statment if you are using the cloud environment by Cousera and commented out the other one.

In [ ]:
"""Get highest paid employee. Latest version of mysql returns an eror"""
#HighestSalaryEmployee  = """select Name, Role, max(Annual_Salary) from Employees"""
HighestSalaryEmployee  = """select Name, Role, max(Annual_Salary) from Employees group by EmployeeID limit 1"""

In [ ]:
#Execute select statement for highest paid employee
cursor.execute(HighestSalaryEmployee)

In [ ]:
#get results fro cursor
results = cursor.fetchall()

In [ ]:
#display results
for data in results:
    print(f"""
    Employee with Highest Salary
    Name: {data[0]}
    Role: {data[1]}
    """)

In [ ]:
#Get detials of guests who booked between 18 and 20
NumberOfGuestBetweenTime = """select count(Hour(BookingSlot)) from Bookings where Hour(BookingSlot) between 18 and 20"""

In [ ]:
#Execute guest booking time statements
cursor.execute(NumberOfGuestBetweenTime)

In [ ]:
#get results from cursor
results = cursor.fetchall()

In [ ]:
#Output results
for data in results:
    print(f"""
    Number of guests booked between 18:00 and 20:00
    Number: {data[0]}
    """)

In [ ]:
#Get details of guest
DetailsOfGuests = """select concat(GuestFirstName, ' ', GuestLastName) as GuestFullName, BookingID  from Bookings order by BookingSlot"""

In [ ]:
#Execute detials of guest statement
cursor.execute(DetailsOfGuests)

In [ ]:
#get results from cursor
results = cursor.fetchall()

In [ ]:
#display results
print(" Details of Guest with sorted with respect to their BookingSlot")
for data in results:
    print(f"""
    Guest Full Name: {data[0]}
    BookingID: {data[1]}
    """)

In [ ]:
print("Return connection to pool")
task3connection.close()
print("Connection returned to pool successfully")

# Task 4

Task 4:
Create a stored procedure named BasicSalesReport that returns the following statistics: 

Total sales

Average sale

Minimum bill paid

Maximum bill paid


In [ ]:
"""Get a connection from connection pool for task 4"""
task4connection = pool.get_connection()
if task4connection.is_connected:
    print("Connected successfully")
else:
    print("Error getting connection to pool")

In [ ]:
#Create a cursor for task4
cursor = task4connection.cursor()

In [ ]:
#Sales procedure statment
BasicSalesProcedure = """
create procedure BasicSalesReport()
begin
select sum(BillAmount) as Total_Sales, avg(BillAmount) as Average_Sale, min(BillAmount) as Minimum_Bill_Paid
, max(BillAmount) as Maximum_Bill_Paid from Orders;
end;
"""

In [ ]:
#Execute procedure creation statment
cursor.execute(BasicSalesProcedure)

In [ ]:
#Call Procedure
cursor.callproc("BasicSalesReport")

In [ ]:
#Get procedure call results from cursor
results = next(cursor.stored_results())

In [ ]:
#Store procedure results in a dataset
dataset = results.fetchall()

In [ ]:
#Display results
for data in dataset:
    print(f"""
    Total Sales: {data[0]}
    Average Sale: {data[1]}
    Minimum Bill Paid: {data[2]}
    Maximum Bill Paid: {data[3]}
    
    """)

In [ ]:
#Return connection to connection pool
print("Returing connection to cursor")
task4connection.close()
print("Connection return successfully")

# Task 5

In [ ]:
"""Get connection pool for task 4"""
print("Get connection")
task5 = pool.get_connection()
if task5.is_connected:
    print("Connected Successfully")
else:
    print("An error occured")

In [ ]:
#Create a buffered cursor for task5
cursor = task5.cursor(buffered=True)

In [ ]:
#Details statment
stmet = """
select b.BookingSlot, concat(b.GuestFirstName, ' ', b.GuestLastName) as GuestFullName, e.Name, e.Role from Bookings as b
inner join Employees as e on b.EmployeeID = e.EmployeeID
"""

In [ ]:
#Execute sql statement
cursor.execute(stmet)

In [ ]:
#get results from cursor
result = cursor.fetchmany(size=3)

In [ ]:
#display results
for data in result:
    print(f"""Booking Slot: {data[0]}
            GuestName: {data[1]}
            Assigned to: {data[2]} [{data[3]}]
          """)

In [ ]:
#Return connection to connection pool
print("Returning connections")
task5.close()
print("Connection closed")